In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

# Load NLTK stopwords
stop_words = set(stopwords.words('english'))

# Define preprocessing function
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove stop words
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Define GCN model with GloVe embeddings
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, embedding_matrix):
        super(GCN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=True)
        self.fc1 = nn.Linear(embedding_matrix.shape[1], hidden_dim)  # Adjust input dimension here
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x, dim=1)  # Average pooling over embeddings
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Load and preprocess data
train_data = pd.read_csv('/content/drive/MyDrive/restrain.csv')
test_data = pd.read_csv('/content/drive/MyDrive/restest.csv')

# Drop rows with missing values
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

# Apply text preprocessing to the data
train_data['Sentence'] = train_data['Sentence'].apply(preprocess_text)
test_data['Sentence'] = test_data['Sentence'].apply(preprocess_text)

# Initialize CountVectorizer with text preprocessing
vectorizer = CountVectorizer()

# Fit and transform on the training data
X_train = vectorizer.fit_transform(train_data['Sentence']).toarray()
# Transform the test data
X_test = vectorizer.transform(test_data['Sentence']).toarray()

# Encode labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_data['Polarity'])
y_test = encoder.transform(test_data['Polarity'])

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Prepare word index
def prepare_embedding_matrix(embeddings, word_index, embedding_dim):
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for word, i in word_index.items():
        if i < num_words:
            embedding_vector = embeddings.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Load GloVe embeddings
glove_embeddings = load_glove_embeddings('/content/drive/MyDrive/glove.6B.100d.txt')

# Prepare word index
word_index = {word: index for index, word in enumerate(vectorizer.get_feature_names_out())}

# Prepare embedding matrix
embedding_dim = len(next(iter(glove_embeddings.values())))
embedding_matrix = prepare_embedding_matrix(glove_embeddings, word_index, embedding_dim)

# Convert data to PyTorch tensors
X_train = torch.LongTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.LongTensor(X_test)
y_test = torch.LongTensor(y_test)

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create datasets and data loaders
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=200, shuffle=False)

# Define training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Define evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            total_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            predictions.extend(pred.cpu().numpy())
            ground_truths.extend(target.cpu().numpy())
    accuracy = correct / len(test_loader.dataset)
    avg_loss = total_loss / len(test_loader)
    return avg_loss, accuracy, predictions, ground_truths

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model, optimizer, and loss function
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = len(encoder.classes_)
model = GCN(input_dim, hidden_dim, output_dim, embedding_matrix).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Train and evaluate the model
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy, predictions, ground_truths = evaluate(model, test_loader, criterion, device)
    print(f'Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    # Calculate precision, recall, and F1 score
    precision = precision_score(ground_truths, predictions, average='weighted')
    recall = recall_score(ground_truths, predictions, average='weighted')
    f1 = f1_score(ground_truths, predictions, average='weighted')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

# Save trained model
torch.save(model.state_dict(), 'gcn_model.pth')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Epoch 1/100, Train Loss: 0.9818, Test Loss: 0.9136, Test Accuracy: 0.6497
Precision: 0.4221, Recall: 0.6497, F1 Score: 0.5117


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/100, Train Loss: 0.9389, Test Loss: 0.9193, Test Accuracy: 0.6497
Precision: 0.4221, Recall: 0.6497, F1 Score: 0.5117


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 98/100, Train Loss: 0.9551, Test Loss: 0.9114, Test Accuracy: 0.6497
Precision: 0.4221, Recall: 0.6497, F1 Score: 0.5117
Epoch 99/100, Train Loss: 0.9461, Test Loss: 0.9141, Test Accuracy: 0.6497
Precision: 0.4221, Recall: 0.6497, F1 Score: 0.5117


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 100/100, Train Loss: 0.9451, Test Loss: 0.9108, Test Accuracy: 0.6497
Precision: 0.4221, Recall: 0.6497, F1 Score: 0.5117


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')